In [ ]:
import json
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont, QPixmap
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QPushButton, QVBoxLayout, QWidget,
    QLabel, QStackedWidget, QHBoxLayout, QSlider, QDialog, QLineEdit, QComboBox,
    QScrollArea, QSizePolicy
)
from keras.preprocessing import image

In [ ]:
print(sys.executable)
print(sys.version)

In [ ]:
nutrition = pd.read_csv('nutr_df.csv')
display(nutrition.head())
print("num_of_nutrition_recipes",nutrition.shape[0])
nutrition = nutrition[['recipe_id', 'protein', 'fat', 'carbohydrates']]
nutrition = nutrition.set_index('recipe_id')
nutrition = nutrition.dropna()
display(nutrition.head())

In [ ]:
def load_train_data(file):
    data = np.genfromtxt(file, delimiter=',', dtype=int, skip_header=1)
    user_ids = np.unique(data[:, 0])
    item_ids = np.unique(data[:, 1])
    # create dictionaries for user and item ids to their indeces in the matrix
    user_indeces = {user_id: index for index, user_id in enumerate(user_ids)}
    item_indeces = {item_id: index for index, item_id in enumerate(item_ids)}
    n_users = user_ids.size
    n_items = item_ids.size

    # Create the user-item matrix
    UI_matrix = np.zeros((n_users, n_items))
    for row in data:
        user_id, item_id, rating = row
        UI_matrix[user_indeces[user_id], item_indeces[item_id]] = rating

    return UI_matrix, user_indeces, item_indeces

uiMatrix, userindices, recipe_id_to_index = load_train_data(file='reduced_train.csv')
print("Training UI matrix size",uiMatrix.shape)
print("number of recipes in matrix",len(recipe_id_to_index))
# get ratings for first user
user_ratings = uiMatrix[1]
rated_recipe_indices = np.where(user_ratings > 0)[0]

# Create a reverse mapping from index to item ID
index_to_recipe_id = {index: item_id for item_id, index in recipe_id_to_index.items()}
rated_recipe_ids = [index_to_recipe_id[i] for i in rated_recipe_indices]
print(rated_recipe_ids)


In [ ]:

recipe_meta = pd.read_csv('recipe_meta.csv')
recipe_meta_df = recipe_meta.set_index('recipe_id')


In [ ]:

class InputDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("Enter Details")

        self.layout = QVBoxLayout()

        self.age_label = QLabel("Enter your age:")
        self.age_input = QLineEdit()
        self.layout.addWidget(self.age_label)
        self.layout.addWidget(self.age_input)

        self.gender_label = QLabel("Enter your gender (male/female):")
        self.gender_input = QLineEdit()
        self.layout.addWidget(self.gender_label)
        self.layout.addWidget(self.gender_input)

        self.height_label = QLabel("Enter your height (in cm):")
        self.height_input = QLineEdit()
        self.layout.addWidget(self.height_label)
        self.layout.addWidget(self.height_input)

        self.weight_label = QLabel("Enter your weight (in kg):")
        self.weight_input = QLineEdit()
        self.layout.addWidget(self.weight_label)
        self.layout.addWidget(self.weight_input)

        self.activity_level_label = QLabel("How would you rate your activity level on a scale from 1 to 5? \n" 
                                           "1. Sedentary: little or no exercise \n"
                                           "2. Exercise 1-3 times/week \n"
                                           "3. Exercise 4-5 times/week \n"
                                           "4. Daily exercise or intense exercise 3-4 times/week \n"
                                           "5. Intense exercise 6-7 times/week")
        self.activity_level_input = QComboBox()
        self.activity_level_input.addItems(["1", "2", "3", "4", "5"])
        self.layout.addWidget(self.activity_level_label)
        self.layout.addWidget(self.activity_level_input)

        self.goal_label = QLabel("Please indicate your weight goal (weekly): \n" 
                                 "1. Maintain weight \n"
                                 "2. Mild weight loss (0-1% body weight)\n"
                                 "3. Moderate weight loss (1-3% body weight) \n"
                                 "4. Extreme weight loss (>3% body weight)\n"
                                 "5. Mild weight gain (0-1% body weight)\n"
                                 "6. Moderate weight gain (1-3% body weight)\n"
                                 "7. Extreme weight gain (>3% body weight)\n"
                                 "You can simply type the number corresponding to your goal.")
        self.goal_input = QComboBox()
        self.goal_input.addItems(["1", "2", "3", "4", "5", "6", "7"])
        self.layout.addWidget(self.goal_label)
        self.layout.addWidget(self.goal_input)
        
       
        self.submit_button = QPushButton("Submit")
        self.submit_button.setStyleSheet("color: black;")
        self.submit_button.clicked.connect(self.store_values)
        self.layout.addWidget(self.submit_button)
        self.setLayout(self.layout)
        self.submit_button.clicked.connect(self.quit_application)
        
        

    def store_values(self):
        self.age = self.age_input.text()
        self.gender = self.gender_input.text()
        self.height = self.height_input.text()
        self.weight = self.weight_input.text()

        activity_level_mapping = {
            "1": "2",
            "2": "3",
            "3": "4",
            "4": "5",
            "5": "6"
        }
        self.activity_level = activity_level_mapping.get(self.activity_level_input.currentText(), "Invalid input")

        goal_mapping = {
            "1": "maintain",
            "2": "mildlose",
            "3": "weightlose",
            "4": "extremelose",
            "5": "mildgain",
            "6": "weightgain",
            "7": "extremegain"
        }
        self.goal = goal_mapping.get(self.goal_input.currentText(), "maintain")

        self.accept()


    def quit_application(self):
        QApplication.quit()

    def closeEvent(self, event):
        self.store_values()  
        super().closeEvent(event)


    def get_age(self):
            return self.age

    def get_gender(self):
        return self.gender

    def get_height(self):
        return self.height

    def get_weight(self):
        return self.weight

    def get_activity_level(self):
        return self.activity_level

    def get_goal(self):
        return self.goal

if __name__ == "__main__":
    app = QApplication([])
    dialog = InputDialog()
    dialog.show()
    result = dialog.exec_()
    if result == QDialog.Accepted:
        age = dialog.get_age()
        gender = dialog.get_gender()
        height = dialog.get_height()
        weight = dialog.get_weight()
        activity_level = dialog.get_activity_level()
        goal = dialog.get_goal()
        print(f"Age: {age}")
        print(f"Gender: {gender}")
        print(f"Height: {height}")
        print(f"Weight: {weight}")
        print(f"Activity Level: {activity_level}")
        print(f"Goal: {goal}")
    else:
        sys.exit('Dialog closed, stopping notebook')
       
        

In [ ]:

url = "https://fitness-calculator.p.rapidapi.com/macrocalculator"

querystring = {"age":age,"gender":gender,"height":height,"weight":weight,"activitylevel":activity_level,"goal":goal}

load_dotenv()  # take the API Key  from .env.
headers = {
    "X-RapidAPI-Key": os.getenv("RAPIDAPI_KEY"),
    "X-RapidAPI-Host": "fitness-calculator.p.rapidapi.com"
}
response = requests.get(url, headers=headers, params=querystring)

print(response.json())

In [ ]:
original_data = response.json()
print(original_data)
transformed_data = {
    'data': {
        diet: {
            nutrient: f"{round(value)}g"
            for nutrient, value in nutrients.items()
        }
        for diet, nutrients in original_data['data'].items() if diet != 'calorie'  
}
print(transformed_data)
print(original_data)

In [ ]:
%gui qt

In [ ]:

class SliderWithLabels(QWidget):
    def __init__(self, title, initial_value, parent=None):
        super().__init__(parent)
        layout = QHBoxLayout()
        
        self.lower_label = QLabel(str(int(0.8 * initial_value)))
        self.upper_label = QLabel(str(int(1.2 * initial_value)))
        self.value_label = QLabel(f'{title}: {initial_value}g')
        
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(int(0.8 * initial_value))
        self.slider.setMaximum(int(1.2 * initial_value))
        self.slider.setValue(initial_value)
        
        self.slider.valueChanged.connect(self.update_value_label)

        layout.addWidget(self.lower_label)
        layout.addWidget(self.slider)
        layout.addWidget(self.upper_label)
        layout.addWidget(self.value_label)
        
        self.setLayout(layout)

    def update_value_label(self, value):
        self.value_label.setText(f'{self.value_label.text().split(":")[0]}: {value}g')
        
  
class MainWindow(QMainWindow):
    def __init__(self, data):

        super().__init__()
        self.nutrient_sliders = {}  
        self.calorie_labels = {}  
        self.data = data
        self.setWindowTitle("Fitness Calculator Results")
        self.nice_font = QFont("Helvetica", 12, QFont.Bold)

        mainLayout = QVBoxLayout()
        mainWidget = QWidget(self)  
        self.setCentralWidget(mainWidget)
        mainWidget.setLayout(mainLayout)

        buttonsLayout = QVBoxLayout()
        mainLayout.addLayout(buttonsLayout) 

        self.central_stack = QStackedWidget()
        mainLayout.addWidget(self.central_stack)  

        colors = {'balanced': '#90ee90', 'lowfat': '#add8e6', 'lowcarbs': '#ffffe0', 'highprotein': '#ffb6c1'}
        self.frames = {}
        self.current_diet = None 

        self.buttons = {}
        diet_display_names = {
            'balanced': 'Balanced Diet',
            'lowfat': 'Low Fat Diet',
            'lowcarbs': 'Low Carbohydrate Diet',
            'highprotein': 'High Protein Diet'
        }
        for diet in ['balanced', 'lowfat', 'lowcarbs', 'highprotein']:
            button = QPushButton(diet_display_names[diet])
            button.setFont(self.nice_font)
            button.clicked.connect(lambda _, d=diet: self.show_diet_details(d))
            buttonsLayout.addWidget(button)

            self.buttons[diet] = button

            frame = QWidget()
            frameLayout = QVBoxLayout(frame)
            frame.setStyleSheet(f"background-color: {colors[diet]}")
            self.nutrient_sliders[diet] = {}
            calories_label = QLabel()
            frameLayout.addWidget(calories_label)
            self.calorie_labels[diet] = calories_label 

            for nutrient, amount in self.data['data'][diet].items():
                amount_value = int(amount.rstrip('g')) 
                nutrient_slider_widget = SliderWithLabels(nutrient.capitalize(), amount_value)
                frameLayout.addWidget(nutrient_slider_widget)
                self.nutrient_sliders[diet][nutrient] = nutrient_slider_widget
                nutrient_slider_widget.slider.valueChanged.connect(lambda _, d=diet: self.update_calories_label(d))

            self.update_calories_label(diet)
            self.central_stack.addWidget(frame)
            self.frames[diet] = frame

        saveMacrosButton = QPushButton("Save Macro targets")
        saveMacrosButton.setFont(self.nice_font)
        saveMacrosButton.clicked.connect(self.save_macros_and_quit)
        mainLayout.addWidget(saveMacrosButton)

    def update_calories_label(self, diet):
        protein = self.nutrient_sliders[diet]['protein'].slider.value()
        carbs = self.nutrient_sliders[diet]['carbs'].slider.value()
        fat = self.nutrient_sliders[diet]['fat'].slider.value()
        total_calories = (protein * 4) + (carbs * 4) + (fat * 9)
        self.calorie_labels[diet].setText(f"Total Calories: {round(total_calories)} kcal")


    def show_diet_details(self, diet):
        frame = self.frames.get(diet)
        if frame:
            self.central_stack.setCurrentWidget(frame)
            self.current_diet = diet  

            
            for button in self.buttons.values():
                button.setStyleSheet("")

          
            self.buttons[diet].setStyleSheet("background-color: #D3D3D3; color: black;")

        else:
            print(f"No frame found for diet: {diet}")

    def quit_application(self):
        QApplication.quit()
        self.close()

    def closeEvent(self, event):
        self.save_slider_values()  
        super().closeEvent(event)

    def save_macros_and_quit(self):
        self.save_slider_values()
        self.close() 

    def save_slider_values(self):
        if self.current_diet is not None:
            diet = self.current_diet
            sliders = self.nutrient_sliders[diet]
            slider_values = {
                diet: {
                    nutrient: slider.slider.value()
                    for nutrient, slider in sliders.items()
                }
            }
            with open('slider_values.json', 'w') as f:
                json.dump(slider_values, f, indent=4)
            print(f"Slider values for {diet} have been saved to slider_values.json")
        else:
            print("No diet was selected; no values saved.")

if __name__ == "__main__":
    app = QApplication.instance() if QApplication.instance() else QApplication([])
    window = MainWindow(transformed_data)
    window.show()
    app.exec_()


# Meal Plan Optimization

# Core GA functions

In [ ]:
# the genome - binary string of 1s and 0s. 
# 1 means the recipe is selected, 0 means it is not selected
# N = number of recipes to select
# fix genome to have N recipes selected 
def repair(child, N):
    selected_indices = np.where(child == 1)[0] #get indices of child where value is 1
    non_selected_indices = np.where(child == 0)[0]
    num_recipes_selected = len(selected_indices)
    # Add recipes if fewer than N are selected
    if num_recipes_selected < N:
        add_indices = np.random.choice(non_selected_indices, size=(N - num_recipes_selected), replace=False)
        child[add_indices] = 1
    # Remove recipes if more than N are selected
    if num_recipes_selected > N:
        swap_out = np.random.choice(selected_indices, size=(num_recipes_selected - N), replace=False)
        child[swap_out] = 0

def generate_random_solution(num_recipes, N):
    solution = np.zeros(num_recipes, dtype=int)
    selected_indices = np.random.choice(num_recipes, N, replace=False)
    solution[selected_indices] = 1
    return solution

# mutate the child by swapping what is in and out of the meal plan (1 or 0 in genome) with a given probability
def mutate(child, num_recipes, N, mutation_rate):
    # Check if a mutation should occur based on the mutation rate
    if np.random.rand() < mutation_rate: #random number is uniformly distributed over 0 and 1
        selected_indices = np.where(child == 1)[0]  
        non_selected_indices = np.where(child == 0)[0]  
        # Choose one recipe to deselect and select
        deselect_index = np.random.choice(selected_indices, 1)[0]
        select_index = np.random.choice(non_selected_indices, 1)[0]
        # Perform the swap
        child[deselect_index] = 0 
        child[select_index] = 1 
    return child

def fitness(solution, recipes, target_macros):
    selected_recipe_indices = np.where(solution == 1)[0]
    selected_recipes = recipes[selected_recipe_indices]
    total_macros = np.sum(selected_recipes, axis=0)  # Sum of macros for selected recipes

    # Calculates percentage deviation for each macronutrient separately
    carbs_deviation = np.abs((total_macros[0] - target_macros[0]) / target_macros[0]) * 100
    protein_deviation = np.abs((total_macros[1] - target_macros[1]) / target_macros[1]) * 100
    fat_deviation = np.abs((total_macros[2] - target_macros[2]) / target_macros[2]) * 100

    # Combine the deviations
    average_percentage_deviation = np.mean([carbs_deviation, protein_deviation, fat_deviation])
    fitness_score = average_percentage_deviation
    return fitness_score

# selects a parent to crossover based on the best fitness score
# tournament size is the number of 'potential' parents to choose from
def tournament_selection(population, fitness_scores, tournament_size):
    selected_indices = np.random.choice(np.arange(len(population)), tournament_size)
    selected_indices = selected_indices.flatten()  # Ensure that selected_indices is a 1D array
    best_index = selected_indices[np.argmin(fitness_scores[selected_indices])]
    return population[best_index]

# single point crossover
def single_point_crossover(parent1, parent2, N):
    crossover_point = np.random.randint(len(parent1))
    child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    repair(child, N)
    return child

# uniform crossover
# Each gene in the child solution has an equal probability (50%) of being inherited from either parent
def uniform_crossover(parent1, parent2, N):
    child = np.array([parent1[i] if np.random.rand() < 0.5 else parent2[i] for i in range(len(parent1))])
    repair(child, N)
    return child

# P- GA 

In [ ]:

def tournament_selection_p(population, fitness_scores, protein_diffs, tournament_size, protein_diffs_weight, N):
    selected_indices = np.random.choice(np.arange(len(population)), tournament_size)
    selected_indices = selected_indices.flatten()  # Ensure that selected_indices is a 1D array
    combined_scores = []
    for index in selected_indices:
        solution = population[index]
        solution_protein_diffs = protein_diffs[solution.astype(bool)]
        combined_score = (fitness_scores[index] * (1-protein_diffs_weight)) + ((np.sum(solution_protein_diffs)/N) * protein_diffs_weight)
        combined_scores.append(combined_score)

    best_index = selected_indices[np.argmin(combined_scores)]
    return population[best_index].copy()  

def genetic_algo_P(recipe_macros, target_macros,  population_size, num_generations, mutation_rate, patience, deviation_threshold, tournament_size, crossover_function, N=5, p_diff_weight=0.3):
    num_recipes = recipe_macros.shape[0]
    population = [generate_random_solution(num_recipes, N) for _ in range(population_size)]
    # population = [generate_random_solution_protein(num_recipes, N, recipe_macros, target_macros, p_random_weight) for _ in range(population_size)]
    best_fitness = np.inf  # Initialize best_fitness
    patience_counter = 0
    new_population_counter = 0

    # Calculate protein differences for all recipes
    protein_diffs = np.zeros(num_recipes, dtype=float)
    for i in range(num_recipes):
        protein_diffs[i] = np.abs((recipe_macros[i][0] - (target_macros[0]/N)) / (target_macros[0]/N)) * 100

    for generation in range(num_generations):
        fitness_scores = np.array([fitness(solution, recipe_macros, target_macros) for solution in population]).flatten()
        percentage_deviation = np.min(fitness_scores)
        
        # If the percentage deviation is less than the threshold, stop training
        if percentage_deviation < deviation_threshold:
            print("Percentage deviation is less than threshold")
            break

        # If the fitness is not improving
        if percentage_deviation >= best_fitness:
            patience_counter += 1
            new_population_counter += 1
            # If the fitness hasn't improved for 'patience' generations, stop training
            if patience_counter >= patience:
                print(f"Fitness hasn't improved for {patience} generations, stop training")
                break
        else:
            best_fitness = percentage_deviation
            patience_counter = 0
            new_population_counter = 0

        new_population = []
        # select pairs of parents to crossover
        for _ in range(population_size):
            parent1 = tournament_selection_p(population, fitness_scores, protein_diffs, tournament_size, p_diff_weight, N)
            parent2 = tournament_selection_p(population, fitness_scores, protein_diffs, tournament_size, p_diff_weight, N)
            
            child = crossover_function(parent1, parent2, N)
            child = mutate(child, num_recipes, N, mutation_rate)
           
            new_population.extend([child])
        
        population = new_population
    
    # Recalculate fitness scores for the final generation
    fitness_scores = np.array([fitness(solution, recipe_macros, target_macros) for solution in population]).flatten()
    best_index = np.argmin(fitness_scores)
    # Return the best solution from the final population
    return population[best_index]

# PC - GA 

In [ ]:
def genetic_algo_PC(recipe_macros, target_macros,  population_size, num_generations, mutation_rate, patience, deviation_threshold, tournament_size, crossover_function, N=3, p_diff_weight=0.3):
    num_recipes = recipe_macros.shape[0]
    population = [generate_random_solution(num_recipes, N) for _ in range(population_size)]
    # population = [generate_random_solution_protein(num_recipes, N, recipe_macros, target_macros, p_random_weight) for _ in range(population_size)]
    best_fitness = np.inf  # Initialize best_fitness
    patience_counter = 0
    new_population_counter = 0

    p_and_carb_diffs = np.zeros(num_recipes, dtype=float)  # Create a 2D array to store both protein and carb diffs
    for i in range(num_recipes):
        protein_diff = np.abs((recipe_macros[i][0] - (target_macros[0]/N)) / (target_macros[0]/N)) * 100
        carb_diff = np.abs((recipe_macros[i][2] - (target_macros[2]/N)) / (target_macros[2]/N)) * 100
        p_and_carb_diffs[i] = (carb_diff+protein_diff)/2  # Store both diffs in the array
        
    for generation in range(num_generations):
        fitness_scores = np.array([fitness(solution, recipe_macros, target_macros) for solution in population]).flatten()
        percentage_deviation = np.min(fitness_scores)
        
        # If the percentage deviation is less than the threshold, stop training
        if percentage_deviation < deviation_threshold:
            print("Percentage deviation is less than threshold")
            break

        # If the fitness is not improving
        if percentage_deviation >= best_fitness:
            patience_counter += 1
            new_population_counter += 1
            # If the fitness hasn't improved for 'patience' generations, stop training
            if patience_counter >= patience:
                print(f"Fitness hasn't improved for {patience} generations, stop training")
                break
        else:
            best_fitness = percentage_deviation
            patience_counter = 0
            new_population_counter = 0

        new_population = []
        # select pairs of parents to crossover
        for _ in range(population_size):
            parent1 = tournament_selection_p(population, fitness_scores, p_and_carb_diffs, tournament_size, p_diff_weight, N)
            parent2 = tournament_selection_p(population, fitness_scores, p_and_carb_diffs, tournament_size, p_diff_weight, N)
            
            child = crossover_function(parent1, parent2, N)
            child = mutate(child, num_recipes, N, mutation_rate)
           
            new_population.extend([child])
        
        population = new_population
    
    # Recalculate fitness scores for the final generation
    fitness_scores = np.array([fitness(solution, recipe_macros, target_macros) for solution in population]).flatten()
    best_index = np.argmin(fitness_scores)
    # Return the best solution from the final population
    return population[best_index]

Convert target macros to 1D Array

In [ ]:
# Load the JSON file storing the chosen macro targets 
with open('slider_values.json', 'r') as f:
    data = json.load(f)

# extract the diet category the users targets are in
keys = list(data.keys())
diet_type = (keys[0])  # balanced, lowfat, lowcarbs, highprotein

macro_values = next(iter(data.values()))
macro_targets = np.array([macro_values['protein'], macro_values['fat'], macro_values['carbs']])

Convert recipe samples to 2D array to match macro targets

In [ ]:
# check nutrition and uiMatrix have all the same recipe_ids
nutrition_recipe_ids = set(nutrition.index)
uiMatrix_recipe_ids = set(recipe_id_to_index.keys())
are_equal = nutrition_recipe_ids == uiMatrix_recipe_ids
print("Do nutrition and uiMatrix have all the same recipe_ids?", are_equal)
if not are_equal:
    nutrition_not_uiMatrix = nutrition_recipe_ids - uiMatrix_recipe_ids
    uiMatrix_not_nutrition = uiMatrix_recipe_ids - nutrition_recipe_ids
    print("Recipe_ids in nutrition but not in uiMatrix:", nutrition_not_uiMatrix)
    print("Recipe_ids in uiMatrix but not in nutrition:", uiMatrix_not_nutrition)

In [ ]:

usersamples = pd.read_csv('all_user_ids.csv')
demo_user = usersamples['all'].values[6]
demo_user_index = userindices[demo_user]
demo_user_ratings = uiMatrix[demo_user_index]

# get the indices of the recipes that the demo_user has rated
rated_recipe_indices = np.where(demo_user_ratings > 0)[0]

# get the ratings the demo_user gave to these recipes
recipe_ratings = demo_user_ratings[rated_recipe_indices]
recipe_ratings = np.round(recipe_ratings).astype(int)


index_to_recipe_id = {index: item_id for item_id, index in recipe_id_to_index.items()}
rated_recipe_ids = [index_to_recipe_id[i] for i in rated_recipe_indices if i in index_to_recipe_id]
print("Number of rated recipes for user:", len(rated_recipe_ids))
recipe_visualisation = pd.DataFrame({'recipe_id': rated_recipe_ids, 'rating': recipe_ratings})
display(recipe_visualisation)
display(nutrition)

In [ ]:
np.set_printoptions(suppress=True)
recipe_samples = nutrition.loc[rated_recipe_ids]
recipe_indices = recipe_samples.index.to_numpy()
recipe_macros = np.round(recipe_samples.values).astype(int)
print("shape of macro targets", macro_targets.shape, "shape of recipe macros",recipe_macros.shape, "shape of recipe_ratings", recipe_ratings.shape)
r_df = pd.DataFrame(recipe_macros, columns=['protein', 'fat', 'carbohydrates'], index=recipe_indices)
display(r_df)

- Include ratings in fitness function
- when selecting parents for crossover, select based on fitness score and rating
- calculate rating score using method in recipe recommender
- choose parent based on a combination of fitness score and rating score (e.g. 70% fitness score, 30% rating score) allow user to adjust this ratio in the GUI


In [ ]:

def fitness_and_rating(solution, recipe_macros, target_macros, recipe_ratings):
    selected_recipe_indices = np.where(solution == 1)[0]
    selected_recipes = recipe_macros[selected_recipe_indices]
    total_macros = np.sum(selected_recipes, axis=0)
    euclidean_distance = np.linalg.norm(total_macros - target_macros)
    target_sum = np.sum(target_macros)
    percentage_deviation = (euclidean_distance / target_sum) * 100

    # Calculate total rating
    selected_ratings = recipe_ratings[selected_recipe_indices]
    total_rating = np.sum(selected_ratings)

    return percentage_deviation, total_rating

the normalized_rating_scores are subtracted from the normalized_fitness_scores when calculating the combined_scores. This means that individuals with higher ratings will have smaller combined scores, and since the function selects the individual with the smallest combined score, individuals with higher ratings are more likely to be selected.

In [ ]:
# built on top of tournament_selection_p (diet specialized tournament selection)
def rating_informed_tournament_selection(population, fitness_scores, protein_diffs, rating_scores, tournament_size, protein_diffs_weight, N, rating_weight):
    selected_indices = np.random.choice(np.arange(len(population)), tournament_size)
    selected_indices = selected_indices.flatten()
    
    # Calculate fitness_scores using the strategy of the tournament_selection_p function
    combined_scores = []
    for index in selected_indices:
        solution = population[index]
        solution_protein_diffs = protein_diffs[solution.astype(bool)]
        combined_score = (fitness_scores[index] * (1-protein_diffs_weight)) + ((np.sum(solution_protein_diffs)/N) * protein_diffs_weight)
        combined_scores.append(combined_score)
    
    rating_scores = rating_scores[selected_indices]

    # Normalize rating scores to be between 0 and 1
    
    max_rating_score = np.max(rating_scores)
    min_rating_score = np.min(rating_scores)
    if max_rating_score == min_rating_score:
        normalized_rating_scores = np.zeros_like(rating_scores) #all solutions have same rating total so dont want to divide by 0
    else:
        normalized_rating_scores = (rating_scores - min_rating_score) / (max_rating_score - min_rating_score)

    # Normalize combined scores to be between 0 and 1
    max_combined_score = np.max(combined_scores)
    min_combined_score = np.min(combined_scores)
    if max_combined_score == min_combined_score:
        normalized_combined_scores = np.zeros_like(combined_scores) #all solutions have same combined fitness score (unlikely)
    else:
        normalized_combined_scores = (combined_scores - min_combined_score) / (max_combined_score - min_combined_score)
    
    final_scores = ((1 - rating_weight) * normalized_combined_scores) - (rating_weight * normalized_rating_scores)

    best_index = selected_indices[np.argmin(final_scores)]  # Select the individual with the smallest combined score
    return population[best_index]


# TSI Injection (Best injection strategy found from testing)

In [ ]:
def hybrid_genetic_algo_P(recipe_macros, target_macros, recipe_ratings, rating_weight, population_size, num_generations, mutation_rate, patience, deviation_threshold, tournament_size, crossover_function, N=5, p_diff_weight = 0.3):
    num_recipes = recipe_macros.shape[0]
    population = [generate_random_solution(num_recipes, N) for _ in range(population_size)]
    best_fitness = np.inf  # Initialize best_fitness
    patience_counter = 0
    new_population_counter = 0

    # Calculate protein differences for all recipes
    protein_diffs = np.zeros(num_recipes, dtype=float)
    for i in range(num_recipes):
        protein_diffs[i] = np.abs((recipe_macros[i][0] - (target_macros[0]/N)) / (target_macros[0]/N)) * 100

    for generation in range(num_generations):
        # Calculate fitness and rating scores
        fitness_and_rating_scores = np.array([fitness_and_rating(solution, recipe_macros, target_macros, recipe_ratings) for solution in population])
        fitness_scores = fitness_and_rating_scores[:, 0]
        rating_scores = fitness_and_rating_scores[:, 1]

        percentage_deviation = np.min(fitness_scores)
        
        # If the percentage deviation is less than the threshold, stop training
        if percentage_deviation < deviation_threshold:
            print("Percentage deviation is less than threshold")
            break

        # If the fitness is not improving
        if percentage_deviation >= best_fitness:
            patience_counter += 1
            new_population_counter += 1
            # If the fitness hasn't improved for 'patience' generations, stop training
            if patience_counter >= patience:
                print(f"Fitness hasn't improved for {patience} generations, stop training")
                break
        else:
            best_fitness = percentage_deviation
            patience_counter = 0
            new_population_counter = 0

        new_population = []
        # select pairs of parents to crossover
        for _ in range(population_size):
            parent1 = rating_informed_tournament_selection(population, fitness_scores, protein_diffs, rating_scores, tournament_size, p_diff_weight, N, rating_weight)
            parent2 = rating_informed_tournament_selection(population, fitness_scores, protein_diffs, rating_scores, tournament_size, p_diff_weight, N, rating_weight)
            
            child = crossover_function(parent1, parent2, N)
            child = mutate(child, num_recipes, N, mutation_rate)
           
            new_population.extend([child])
        
        population = new_population
    
    # Recalculate fitness scores for the final generation
    fitness_scores = np.array([fitness(solution, recipe_macros, target_macros) for solution in population]).flatten()
    best_index = np.argmin(fitness_scores)
    # Return the best solution from the final population
    return population[best_index]

In [ ]:
def hybrid_genetic_algo_PC(recipe_macros, target_macros, recipe_ratings, rating_weight, population_size, num_generations, mutation_rate, patience, deviation_threshold, tournament_size, crossover_function, N=3, p_diff_weight = 0.3):
    num_recipes = recipe_macros.shape[0]
    population = [generate_random_solution(num_recipes, N) for _ in range(population_size)]
    best_fitness = np.inf 
    patience_counter = 0
    new_population_counter = 0

    # Calculate protein differences for all recipes
    p_and_carb_diffs = np.zeros(num_recipes, dtype=float)  # Creates a 2D array to store both protein and carb diffs
    for i in range(num_recipes):
        protein_diff = np.abs((recipe_macros[i][0] - (target_macros[0]/N)) / (target_macros[0]/N)) * 100
        carb_diff = np.abs((recipe_macros[i][2] - (target_macros[2]/N)) / (target_macros[2]/N)) * 100
        p_and_carb_diffs[i] = (carb_diff+protein_diff)/2  # Store both diffs in the array

    for generation in range(num_generations):
        # Calculate fitness and rating scores
        fitness_and_rating_scores = np.array([fitness_and_rating(solution, recipe_macros, target_macros, recipe_ratings) for solution in population])
        fitness_scores = fitness_and_rating_scores[:, 0]
        rating_scores = fitness_and_rating_scores[:, 1]

        percentage_deviation = np.min(fitness_scores)
        
        # If the percentage deviation is less than the threshold, stop training
        if percentage_deviation < deviation_threshold:
            print("Percentage deviation is less than threshold")
            break

        # If the fitness is not improving
        if percentage_deviation >= best_fitness:
            patience_counter += 1
            new_population_counter += 1
            if patience_counter >= patience:
                print(f"Fitness hasn't improved for {patience} generations, stop training")
                break
        else:
            best_fitness = percentage_deviation
            patience_counter = 0
            new_population_counter = 0

        new_population = []
        
        for _ in range(population_size):
            parent1 = rating_informed_tournament_selection(population, fitness_scores, p_and_carb_diffs, rating_scores, tournament_size, p_diff_weight, N, rating_weight)
            parent2 = rating_informed_tournament_selection(population, fitness_scores, p_and_carb_diffs, rating_scores, tournament_size, p_diff_weight, N, rating_weight)
            
            child = crossover_function(parent1, parent2, N)
            child = mutate(child, num_recipes, N, mutation_rate)
           
            new_population.extend([child])
        
        population = new_population
    
    # Recalculate fitness scores for the final generation
    fitness_scores = np.array([fitness(solution, recipe_macros, target_macros) for solution in population]).flatten()
    best_index = np.argmin(fitness_scores)
    # Return the best solution from the final population
    return population[best_index]

In [ ]:

class RatingWeightDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Taste Preferences vs Nutritional Goals")

        layout = QVBoxLayout()

        self.label = QLabel("How much weighting would you like to give to your taste preferences?")
        layout.addWidget(self.label)

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(100)
        self.slider.valueChanged.connect(self.slider_value_changed)
        layout.addWidget(self.slider)

        self.value_label = QLabel("0%")
        self.value_label.setFont(QFont("Arial", 14, QFont.Bold))
        layout.addWidget(self.value_label)

        self.note_label = QLabel("(PS: 100% means you give 0% importance to achieving your nutritional goals and vice versa)")
        layout.addWidget(self.note_label)

        self.submit_button = QPushButton("Submit")
        self.submit_button.setStyleSheet("color: black")
        self.submit_button.clicked.connect(self.accept)
        layout.addWidget(self.submit_button)

        self.setLayout(layout)

    def slider_value_changed(self, value):
        self.ratingWeight = value / 100.0
        self.value_label.setText(f"{value}%")

    def get_rating_weight(self):
        return self.ratingWeight

def show_display():
    dialog = RatingWeightDialog()
    dialog.exec_()
    return dialog

In [ ]:
dialog = show_display()
ratingWeight = dialog.get_rating_weight()

In [ ]:
ratingWeight = 0

if ratingWeight < 0.1:
    
    if diet_type == 'highprotein' or diet_type == 'lowfat':
        start_time = time.time()
        # recipe_macros, target_macros,  population_size, num_generations, mutation_rate, patience, deviation_threshold, tournament_size, crossover_function
        best_genome = genetic_algo_P(recipe_macros, macro_targets, 200, 400, 0.05, 100, 10, 4, uniform_crossover, 5, 0.3)
        end_time = time.time()
        total_macros = np.sum(recipe_macros[best_genome.astype(bool)], axis=0)
        total_protein, total_fat, total_carb = total_macros
        target_protein, target_fat, target_carb = macro_targets
        print(f"Protein deviation: {np.abs((total_protein - target_protein) / target_protein) * 100:.2f}%, Fat deviation: {np.abs((total_fat - target_fat) / target_fat) * 100:.2f}%, Carb deviation: {np.abs((total_carb - target_carb) / target_carb) * 100:.2f}%,")
        print(f"Average macro deviation: {np.mean([np.abs((total_protein - target_protein) / target_protein), np.abs((total_fat - target_fat) / target_fat), np.abs((total_carb - target_carb) / target_carb)]) * 100:.2f}%")
        print("Time taken for genetic algorithm:", end_time - start_time)

        #recipe_indices is a ordered list of recipe_ids corresponding to the recipe_macro rows
        selected_recipe_indices = recipe_indices[best_genome.astype(bool)]
        print("Best solution genome:", best_genome)
        print("SELECTED RECIPES: ")
        for recipe_id in selected_recipe_indices:
            recipe_name = recipe_meta_df.at[recipe_id, 'recipe_name']
            print(f"~ {recipe_name}")
        print(f"Target protein: {target_protein} Target fat: {target_fat} Target carbs: {target_carb}")
        print(f"Total protein: {total_protein} Total fat: {total_fat} Total carbs: {total_carb}")
          
    else: #lowcarb or balanced
        start_time = time.time()
        best_genome = genetic_algo_PC(recipe_macros, macro_targets, 200, 400, 0.05, 100, 10, 4, uniform_crossover, 3, 0.3)
        end_time = time.time()
        total_macros = np.sum(recipe_macros[best_genome.astype(bool)], axis=0)
        total_protein, total_fat, total_carb = total_macros
        target_protein, target_fat, target_carb = macro_targets
        print(f"Protein deviation: {np.abs((total_protein - target_protein) / target_protein) * 100:.2f}%, Fat deviation: {np.abs((total_fat - target_fat) / target_fat) * 100:.2f}%, Carb deviation: {np.abs((total_carb - target_carb) / target_carb) * 100:.2f}%,")
        print(f"Average macro deviation: {np.mean([np.abs((total_protein - target_protein) / target_protein), np.abs((total_fat - target_fat) / target_fat), np.abs((total_carb - target_carb) / target_carb)]) * 100:.2f}%")
        print("Time taken for genetic algorithm:", end_time - start_time)

        #recipe_indices is a ordered list of recipe_ids corresponding to the recipe_macro rows
        selected_recipe_indices = recipe_indices[best_genome.astype(bool)]
        print("Best solution genome:", best_genome)
        print("SELECTED RECIPES: ")
        for recipe_id in selected_recipe_indices:
            recipe_name = recipe_meta_df.at[recipe_id, 'recipe_name']
            print(f"~ {recipe_name}")
        print(f"Target protein: {target_protein} Target fat: {target_fat} Target carbs: {target_carb}")
        print(f"Total protein: {total_protein} Total fat: {total_fat} Total carbs: {total_carb}")
        for index in selected_recipe_indices:
            display_selected_recipe(index)
            display(recipe_samples.loc[index].transpose())
     
else:     
        
    if diet_type == 'highprotein' or diet_type == 'lowfat':
        start_time = time.time()
        best_genome = hybrid_genetic_algo_P(recipe_macros, macro_targets, recipe_ratings, ratingWeight, 200, 400, 0.05, 100, 10, 4, uniform_crossover)
        end_time = time.time()
        total_macros = np.sum(recipe_macros[best_genome.astype(bool)], axis=0)
        total_protein, total_fat, total_carb = total_macros
        target_protein, target_fat, target_carb = macro_targets
        print(f"Protein deviation: {np.abs((total_protein - target_protein) / target_protein) * 100:.2f}%, Fat deviation: {np.abs((total_fat - target_fat) / target_fat) * 100:.2f}%, Carb deviation: {np.abs((total_carb - target_carb) / target_carb) * 100:.2f}%,")
        print(f"Average macro deviation: {np.mean([np.abs((total_protein - target_protein) / target_protein), np.abs((total_fat - target_fat) / target_fat), np.abs((total_carb - target_carb) / target_carb)]) * 100:.2f}%")
        print("Time taken for genetic algorithm:", end_time - start_time)

        #recipe_indices is a ordered list of recipe_ids corresponding to the recipe_macro rows
        selected_recipe_indices = recipe_indices[best_genome.astype(bool)]
        print("Best solution genome:", best_genome)
        print("SELECTED RECIPES: ")
        for recipe_id in selected_recipe_indices:
            recipe_name = recipe_meta_df.at[recipe_id, 'recipe_name']
            print(f"~ {recipe_name}")
            display(recipe_samples.loc[recipe_id].transpose())
        print(f"Target protein: {target_protein} Target fat: {target_fat} Target carbs: {target_carb}")
        print(f"Total protein: {total_protein} Total fat: {total_fat} Total carbs: {total_carb}")
        
    else: #lowcarb or balanced
        start_time = time.time()
        best_genome = hybrid_genetic_algo_PC(recipe_macros, macro_targets, recipe_ratings, ratingWeight, 200, 400, 0.05, 100, 10, 4, uniform_crossover)
        end_time = time.time()
        total_macros = np.sum(recipe_macros[best_genome.astype(bool)], axis=0)
        total_protein, total_fat, total_carb = total_macros
        target_protein, target_fat, target_carb = macro_targets
        print(f"Protein deviation: {np.abs((total_protein - target_protein) / target_protein) * 100:.2f}%, Fat deviation: {np.abs((total_fat - target_fat) / target_fat) * 100:.2f}%, Carb deviation: {np.abs((total_carb - target_carb) / target_carb) * 100:.2f}%,")
        print(f"Average macro deviation: {np.mean([np.abs((total_protein - target_protein) / target_protein), np.abs((total_fat - target_fat) / target_fat), np.abs((total_carb - target_carb) / target_carb)]) * 100:.2f}%")
        print("Time taken for genetic algorithm:", end_time - start_time)

        #recipe_indices is a ordered list of recipe_ids corresponding to the recipe_macro rows
        selected_recipe_indices = recipe_indices[best_genome.astype(bool)]
        print("Best solution genome:", best_genome)
        print("SELECTED RECIPES: ")
        for recipe_id in selected_recipe_indices:
            recipe_name = recipe_meta_df.at[recipe_id, 'recipe_name']
            print(f"~ {recipe_name}")
            display(recipe_samples.loc[recipe_id].transpose())
    

        print(f"Target protein: {target_protein} Target fat: {target_fat} Target carbs: {target_carb}")
        print(f"Total protein: {total_protein} Total fat: {total_fat} Total carbs: {total_carb}")

In [ ]:
class RecipeDisplay(QWidget):
    def __init__(self, recipe_id, recipe_samples, recipe_meta_df, parent=None):
        super().__init__(parent)
        layout = QVBoxLayout()
        # image_path = f"core-data-images/core-data-images/{recipe_id}.jpg"
        image_path = f"/Users/chloe/Desktop/final_project/core-data-images/core-data-images/{recipe_id}.jpg"
        if not os.path.isfile(image_path):
            print(f"No image file found at {image_path}")
            return

        try:
            pixmap = QPixmap(image_path)
            pixmap = pixmap.scaled(200, 200, Qt.KeepAspectRatio) 
        except IOError:
            print(f"Unable to open image file {image_path}")
            return

        image_label = QLabel()
        image_label.setPixmap(pixmap)

        recipe_name = recipe_meta_df.at[recipe_id, 'recipe_name']
        recipe_info = recipe_samples.loc[recipe_id].transpose()
        protein_info = recipe_info['protein']
        carb_info = recipe_info['carbohydrates']
        fat_info = recipe_info['fat']
        
        info_label = QLabel(f"<b>{recipe_name}</b><br>Protein: {protein_info}<br>Carb: {carb_info}<br>Fat: {fat_info}")

        layout.addWidget(image_label)
        layout.addWidget(info_label)
        
        self.setLayout(layout)

class RecipeDialog(QDialog):
    def __init__(self, selected_recipe_indices, recipe_samples, recipe_meta_df, target_protein, target_fat, target_carb, total_protein, total_fat, total_carb, parent=None):
        super().__init__(parent)
        layout = QVBoxLayout()
        
        title_label = QLabel("<h1 align='center'>Your Personalized Meal Plan</h1>")
        layout.addWidget(title_label)
        
        scrollArea = QScrollArea(self)
        scrollArea.setWidgetResizable(True)
        scrollArea.setSizePolicy(QSizePolicy.Preferred, QSizePolicy.Preferred)
        scrollArea.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)  
        scrollArea.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOn)  
        scrollArea.setStyleSheet("""
            QScrollBar:horizontal {
                border: none;
                background: lightgray;
                height: 10px;
                margin: 0px 22px 0px 22px;
                border-radius: 5px;
            }
            QScrollBar::handle:horizontal {
                background: black;
                min-width: 20px;
                border-radius: 5px;
            }
            QScrollBar::add-line:horizontal {
                border: none;
                background: none;
            }
            QScrollBar::sub-line:horizontal {
                border: none;
                background: none;
            }
        """)
        scrollContent = QWidget(scrollArea)
        scrollLayout = QHBoxLayout(scrollContent)

        for index in selected_recipe_indices:
            recipe_display = RecipeDisplay(index, recipe_samples, recipe_meta_df)
            scrollLayout.addWidget(recipe_display)
        
        scrollArea.setSizePolicy(QSizePolicy.Preferred, QSizePolicy.Preferred)
        scrollArea.setWidget(scrollContent)
        layout.addWidget(scrollArea)

        objectives_label = QLabel("<h2>Nutritional Stats</h2>")
        objectives_label.setAlignment(Qt.AlignCenter)  
        layout.addWidget(objectives_label)


        info_label = QLabel()
        info_label.setAlignment(Qt.AlignCenter) 
        info_label.setTextFormat(Qt.RichText) 

        info_label.setText(f"<p>Total Protein: {total_protein} , Target Protein: {target_protein}</p>"
                   f"<p>Total Fat: {total_fat} , Target Fat: {target_fat}</p>"
                   f"<p>Total Carbs: {total_carb} , Target Carbs: {target_carb}</p>"
                   f"<p></p>")  
        
        layout.addWidget(info_label, alignment=Qt.AlignCenter)

        layout.setContentsMargins(6, 6, 6, 6)  
        self.setLayout(layout)

        self.setSizePolicy(QSizePolicy.Preferred, QSizePolicy.Preferred)  
        self.adjustSize()
        

def show_recipe_dialog(selected_recipe_indices, recipe_samples, recipe_meta_df, target_protein, target_fat, target_carb, total_protein, total_fat, total_carb):
    dialog = RecipeDialog(selected_recipe_indices, recipe_samples, recipe_meta_df, target_protein, target_fat, target_carb, total_protein, total_fat, total_carb)
    dialog.exec_()

In [ ]:
show_recipe_dialog(selected_recipe_indices, recipe_samples, recipe_meta_df, target_protein, target_fat, target_carb, total_protein, total_fat, total_carb)